这个文档得word2Vec model基于[论文](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)。这个模型用来学习词汇的矢量表达，称为"word embeddings"。

## Highlights
本教程旨在强调用TensorFlow创建一个word2vec模型有趣，实质的部分。
* 首先会说明问我们想要用vectors来表达words的动机。
* 查看模型后面的直觉(intuition)和怎么样训练的（用可以很好测量的数学：with a splash of math for measure)
* 在TensorFlow里面怎么样简单的实现模型。
* 最后，怎么样让原始的版本能够延伸得(scale better)更好的办法。
在这个tutorial中会逐步查看code，但是如果说你像直接查看，可以直接查看简单实现[tensorflow/examples/tutorials/word2vec/word2vec_basic.py](https://github.com/tensorflow/tensorflow/blob/r1.5/tensorflow/examples/tutorials/word2vec/word2vec_basic.py)。这个简单例子包含需要下载某些数据的代码，在这个基础上训练，然后可以可视化这个结果。一旦可以查看和运行这个基础的版本，可以查看高阶版本[tutorials/embedding/word2vec.py](https://github.com/tensorflow/models/blob/master/tutorials/embedding/word2vec.py)。在这个版本里面关于比较高阶的TensorFlow的实现，关于怎么样使用thread将数据移动导text model中间，在training的时候怎么样checkpoints等。
但是，首先，让我们查看为什么我们想要学习word embeddings。

## Motivation:Why Learn Word Embeddings?
图像和声音处理系统一般都处理丰富的，高维度的dataset，一般用图像数据原始的像素强度(pixel-intensities)来作为vector编码，或者，例如声音数据的功率谱密度(power spectral density coefficients)系数。对于对象或者语音识别的任务，所有的需要的信息都会很好的在数据中编码（因为人可以很好的从原始的数据中处理这些任务）。但是，自然语言处理系统一般将word认为是分散的原子信号，因此，'cat'可能被表达成Id537,'dog'表达成'Id143'。这些encoding是任意的，关于他们之间的关系不能够提供任何有用的信息。这意味着这些模型当他们学习了'cats'数据，但是在处理'dogs'数据的时候，并不能够提供很大的帮助。代表的words都是唯一的，独立的id导致数据会非常的稀疏，所以这就意味着为了训练统计模型，必须需要更多的数据。使用矢量表达可以战胜这些缺点。
![Title](../../image/audio-image-text.png)

[Vector space model](https://en.wikipedia.org/wiki/Vector_space_model)（VSM）采用连续的矢量空间来表达(embed) words，语义相似的词汇被映射到相近的点。VSMs在NLP中有很长很丰富的例是，但是所有的方法依赖于在[Distributional semantics](https://en.wikipedia.org/wiki/Distributional_semantics#Distributional_Hypothesis)的一些方法，也就是怎么样表示这些在同一个文本中的words共享语义的意思。不同的方法可以提升这个原则的方法可以被分为两大类：count-based method(例如，[latent sematic analysis](https://en.wikipedia.org/wiki/Latent_semantic_analysis)),和predictive methods（例如，[Neural probabilistic language models](http://www.scholarpedia.org/article/Neural_net_language_models)。

这两者的区别在[paper](http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf)详细阐述。概括的说，Count-based methods计算一些词汇在一个大的文本集中和它相邻词汇共同出现的统计次数，然后将这些计算的统计数目每个词汇映射为一个小的，密集(dense)的矢量。Predictive models直接一句学习到的小的，dense embedding vectors(依照模型的参数量）来从它的相邻词汇预测词汇。

Word2vec从原始的文本当中学习word embedding是非常有计算效率的预测模型。它有两种形式，连续的词袋模型(Continuous Bag-of-Word model，CBOW)和Skip-Gram model。在算法上面来说，这些模型都是相似的，除了CBOW从源文本词汇('the cat sits on the')预测目标词汇('mat'),但是skip-gram做相反的事情，它从目标的词汇来预测源文本词汇。这个倒置可能像任意的选择，但是统计上来说，他具有CBOW可以平滑很多分布信息的效果(通过将一个全部的文本认为是一个观察）。大部分情况下，这对于小的dataset来说是十分有效的。但是，skip-gram将每一个文本-目标对(context-target pair)看成是一个新的观察，当dataset更大的时候，它会表现得更好。我们在这个tutorial中会专注于skip-gram模型。

## Scaling up with Noise-Contrastive Training
Nerual probabilistic language models通常是用ML原则来通过给定的前面的words $h$ 来依据softmax函数来最大化下一个word $w_t$(目标）的概率。
\begin{align}
P(w_t | h) &= \text{softmax} (\text{score} (w_t, h)) \\
           &= \frac{\exp \{ \text{score} (w_t, h) \} }
             {\sum_\text{Word w' in Vocab} \exp \{ \text{score} (w', h) \} }
\end{align}
在这里$\text{score} (w_t, h)$计算词汇$w_t$和上下文$h$的兼容性(一般用点乘)。我们通过最大化它的log-likelihood来从training set上面来训练这个模型，也就是说，通过最大化
\begin{align}
 J_\text{ML} &= \log P(w_t | h) \\
  &= \text{score} (w_t, h) -
     \log \left( \sum_\text{Word w' in Vocab} \exp \{ \text{score} (w', h) \} \right).
\end{align}
这位语言模型产生了一个合适的归一化的概率模型。虽然这开销很大(expensive)，因为需要在每个train step计算依据现在的context $h$的在词汇表$V$中的$w'$的score来归一化每个词汇的概率。
![Title](../../image/softmax-nplm.png)
从另一方面来说，对于在word2vec中的特征学习，我们不需要模型的所有的概率。CBOW和skip-gram 模型通过使用一个二分法对象(logistic regression)来在同一个文本中，从$k$ imaginary(noise) words$\tilde w$中区分出实际的target $w_t$。通过下面这个图形来例句CBOW model，一个skip-gram模型就是将下面的模型直接倒过来：
![Title](../../image/nce-nplm.png)
从数学上来说，这个对象（对每个例子）是为了最大化：  
$
J_\text{NEG} = \log Q_\theta(D=1 |w_t, h) +
  k \mathop{\mathbb{E}}_{\tilde w \sim P_\text{noise}}
     \left[ \log Q_\theta(D = 0 |\tilde w, h) \right]
$     
$Q_\theta(D=1 | w, h)$是在这个模型当以及dataset D中从context h中能够看到word w 的二元逻辑回归的概率，根据学习当的embedding vector $\theta$ 计算得到。在实际中，我们通过从noise distribution(也就是说，计算一个[Monte Carlo integration](Monte Carlo integration))当中选取k对比词然后估计他们的期望。
     
这个目标是为了让模型最大化分配高概率给实际的word，低概率给noise words。技术上来说，这叫做Negative Sampling，使用这个loss function是由一个很好的数学原因的：它提出来了一个更加近似softmax函数限制的更新。但是从计算的角度来说更加吸引人，因为这个时候我们只需要选择k个噪声词的比例而不用计算所有的词汇V当中的所有词汇。这也会让模型更容易训练。我们实际上会利用相似噪声对比估计(noise-contrastive estimation,NCE)损失，在这里TensorFlow有一个比较好处理的函数tf.nn.nce_loss()

## The Skip-gram Model
一个示例，查看dataset：  
the quick brown fox jumped over the lazy dog

我们首先在组成这个dataset的词语和它的上下文显示的形式。我们可以定义任何让'context'有意义的形式，事实上，人们会查看语法的上下文（也就是说，当前词汇的语法的依赖,[Levy et al.](https://levyomer.files.wordpress.com/2014/04/dependency-based-word-embeddings-acl-2014.pdf)）。target左边的词汇，target右边的词汇等等。至今为止，注意vanilla 定义和定义'context'作为目标词汇左边和右边的窗口词汇( let's stick to the vanilla definition and define 'context' as the window of words to the left and to the right of a target word)。使用窗口大小为1，然后我们所有的dataset为：  
([the, brown], quick), ([quick, fox], brown), ([brown, jumped], fox), ...
作为(context, target)对。回顾skip-gram是倒置context和target，并且从目标词中想要预测每个context词汇。所以任务变为从 'quick'中预测 'the' and 'brown'，'brown'中预测'quick' and 'fox'等。因此，我们的dataset变为：  
(quick, the), (quick, brown), (brown, quick), (brown, fox), ...  
作为（input，output）对。这个目标方程式通过整个dataset定义的，但是我们一般用SGD来做最优化，也就是说每一次使用一个样本来优化(或者式一个'minibatch' 的__batch_size__的示例，一般而言16 <= batch_size <= 512)。让我们来查看这个步骤的第一步。

首先想象在training step t我们观察到第一个上面的training case， 也就是说从quick中预测the。我们从噪声分布中选择__num_noise__ 数目的noisy(contrastive) 示例，一般而言也就是unigram distribution，$P(w)$。简单而言，我们让__num_noise=1__并且选择__sheep__作为一个noisy示例。然后我们计算观察对和noisy示例的loss。也就是说，在time step t目标变为：  
$J^{(t)}_\text{NEG} = \log Q_\theta(D=1 | \text{the, quick}) +
  \log(Q_\theta(D=0 | \text{sheep, quick}))$
这个目标式为了调整embedding参数$\theta$的值来提升目标方程(在这个例子中，也就是最大化目标方程）。我们通过计算loss关于embedding参数$\theta$的微分，也就是说$\frac{\partial}{\partial \theta} J_\text{NEG}$。然后我们可以通过在梯度上移动一小步来提升embeddings。当这个过程在整个training set上面不断重复，这个过程让每个单词都能够对这个embedding vectors有"滑动"（'moving')的效果，直到这个模型能够从噪声词汇中区分出真实词汇。

我们可以通过将他们通过类似[ t-SNE dimensionality reduction technique](https://lvdmaaten.github.io/tsne/)映射到2维平面进行可视化。当我们查看这些可视化的时候，很显然能够看出这些vector抓住了词汇之间一些普遍的也是非常有用的语义关系以及相关关系。当我们第一次发现在这些产生的向量空间在某些方向表明一些特定的语义关系的时候是非常有趣的，例如：male-female,动词的师太(verb tense)，甚至是国家首都在词汇之间的关系，如下面图中所示(可以查看[Mikolov2013 paper](https://www.aclweb.org/anthology/N13-1090)。
![Title](../../image/linear-relationships.png)
这解释了为什么在非常多典型的(canonical) NLP预测任务当中这些vectors是非常有用的，例如，part-of-speech tagging 或者实体识别(entity recognition)(原始[paper](https://arxiv.org/abs/1103.0398),或后续研究[paper](https://www.aclweb.org/anthology/P10-1040))

## Building the Graph
这都是关于embeddings的，所以，首先我们需要定义embedding matrix。最开始是一个大的random matrix，从一个单位立方体中均匀的初始化这些值：

In [ ]:
embeddings = tf.Variable(tf.random_uniform([covabulary_size,embedding_size],
                                          -1.0,1.0))

noise-contrastive estimation loss(噪声对比估计损失)是从过一个logistic regression来定义的。所以，我们需要定义词汇表中每个词的weights和biases(也可以称为output weights作为input embeddings的反面）。下面的代码定义weights和bias：

In [ ]:
nce_weights = tf.Varaible(tf.truncated_normal([vocabulary_size,embedding_size],
                                             stddev=1.0/math.sqrt(embedding_size)))

nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

现在，我们已经定义好参数，我们可以定义skip-gram model graph。简单而言，让我们假设我们已经将我们的所有文本文集聚集在一个词汇表里面，所以这些文字都可以通过一个整数来表示，查看[code](https://github.com/tensorflow/tensorflow/blob/r1.5/tensorflow/examples/tutorials/word2vec/word2vec_basic.py)中的详细信息。这个skip_gram model止呕两个输入，一个是表示源上下文词汇的整数堆，另外的是目标词汇。首先我们创造作为这些输入的placeholder nodes，这样我们可以随后传入数据：

In [ ]:
# Placeholders for inputs
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size,1])

现在我们需要查看在batch当中的每个源词汇的查看的vector。在这里，可以使用__tf.nn.embedding_lookup__
> embedding_lookup(  
    params,  
    ids,  
    partition_strategy='mod',  
    name=None,  
    validate_indices=True,  
    max_norm=None  
)
在enbedding tensors列表中通过__ids__并行查阅。查阅的tensor列表在参数__params__当中，这个函数是__tf.gather__ 的一般形式，__params__被解释为一个大的embedding tensor的分割。__params__ 也可以时一个__PartitionedVarible__ 通过__tf.get_variable()__ 和一个切分器(partitioner) 返回

> 如果，__len(params)>1__ 根据partition_startegy，ids中的每个元素id将在params元素之间进行切分。在所有的策略中，如果说id space没有均匀切分partitions的数目，每一个第一个(max_id+1)%len(params) partions会被多分配一次id。

> __partition_strategy='mod'__ 将每个id分配给分割(partition) p=id%len(params)。例如，13 ids被分裂成五个分区如下：[[0, 5, 10], [1, 6, 11], [2, 7, 12], [3, 8], [4, 9]]

>  __partition_strategy='div'__ 那么我们将ids用一个连续的形式分配到每一个切分。在这种情况下，13 ids被分裂成五个分区如下：[[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10], [11, 12]]

> lookup的结果会被拼接成一个dense tensor。返回的tensor有的形状时 shape(ids)+shape(params)[1:]

Args:
* params: 表示完整的embedding tensor的单一的tensor，或者是除了第一个维度其他shape都相同的P tensor列表，表示分片的embedding tensors。相应的，一个__PartitionedVariable__，通过依照dimension 0进行切分创造。每个element必须要根据给定的partition_strategy有相应的合适的大小。  
 A single tensor representing the complete embedding tensor, or a list of P tensors all of same shape except for the first dimension, representing sharded embedding tensors. Alternatively, a PartitionedVariable, created by partitioning along dimension 0. Each element must be appropriately sized for the given partition_strategy.


* ids: Tensor，type=int32或者int64包含ids来队params进行查阅  
 A Tensor with type int32 or int64 containing the ids to be looked up in params.


* partition_strategy: 字符串，如果len(params)>1 则相关。默认为'mod'  
 A string specifying the partitioning strategy, relevant if len(params) > 1. Currently "div" and "mod" are supported. Default is "mod".
 
* name: 可选参数  
 A name for the operation (optional).


* validate_indices: DEPRECATED. If this operation is assigned to CPU, values in indices are always validated to be within range. If assigned to GPU, out-of-bound indices result in safe but unspecified behavior, which may include raising an error.


* max_norm: If provided, embedding values are l2-normalized to the value of max_norm.

Returns:
A Tensor with the same type as the tensors in params.

Raises:
ValueError: If params is empty.

问题本质：

线性变换：Z = WX + b

embedding：
输入都是one-hot encoding,所以相当于去除weights矩阵中对应的一行，所以tensorflow封装了这个接口，更加方便的查找了对应的矩阵中的参数。

![Title](../../image/v2-9de68e5c46e9ea1ea480e295b0cc0b87_hd.jpg)

In [ ]:
embed = tf.nn.embedding_lookup(embeddings,train_inputs)

现在就得到了每个词汇的embeddings了，然后可以通过采用noise-contrastive training object来预测目标词
> tf.nn.nce_loss(  
weights,  
    biases,  
    labels,  
    inputs,  
    num_sampled,  
    num_classes,  
    num_true=1,  
    sampled_values=None,  
    remove_accidental_hits=False,  
    partition_strategy='mod',  
    name='nce_loss'  
)
计算和返回noise-contrastive estimation training loss  
__Args:__ 
* weights: A Tensor of shape [num_classes, dim], or a list of Tensor objects whose concatenation along dimension 0 has shape [num_classes, dim]. The (possibly-partitioned) class embeddings.
* biases: A Tensor of shape [num_classes]. The class biases.
* labels: A Tensor of type int64 and shape [batch_size, num_true]. The target classes.
* inputs: A Tensor of shape [batch_size, dim]. The forward activations of the input network.
* num_sampled: An int. The number of classes to randomly sample per batch.
* num_classes: An int. The number of possible classes.
* num_true: An int. The number of target classes per training example.
* sampled_values: a tuple of (sampled_candidates, true_expected_count, sampled_expected_count) returned by a \*_candidate_sampler function. (if None, we default to log_uniform_candidate_sampler)
* remove_accidental_hits: A bool. Whether to remove "accidental hits" where a sampled class equals one of the target classes. If set to True, this is a "Sampled Logistic" loss instead of NCE, and we are learning to generate log-odds instead of log probabilities. See our [Candidate Sampling Algorithms Reference] (https://www.tensorflow.org/extras/candidate_sampling.pdf). Default is False.
* partition_strategy: A string specifying the partitioning strategy, relevant if len(weights) > 1. Currently "div" and "mod" are supported. Default is "mod". See tf.nn.embedding_lookup for more details.
* name: A name for the operation (optional).  
__Returns:__  
* A batch_size 1-D tensor of per-example NCE losses.  
使用这个的一个常见的例子是用这个方法训练，并且计算全部的sigmoid loss作为evaluation或者是inference。再这个情况下，你必须要设置__partition_strategy="div"让这两个loss一直，详细代码如下：
> if mode == "train":  
&ensp;&ensp;  loss = tf.nn.nce_loss(  
&ensp;&ensp;&ensp;&ensp;      weights=weights,  
&ensp;&ensp;&ensp;&ensp;      biases=biases,  
&ensp;&ensp;&ensp;&ensp;      labels=labels,  
&ensp;&ensp;&ensp;&ensp;      inputs=inputs,  
&ensp;&ensp;&ensp;&ensp;      ...,  
&ensp;&ensp; &ensp;&ensp;     partition_strategy="div")  
elif mode == "eval":  
&ensp;&ensp;  logits = tf.matmul(inputs, tf.transpose(weights))  
&ensp;&ensp;  logits = tf.nn.bias_add(logits, biases)  
&ensp;&ensp;  labels_one_hot = tf.one_hot(labels, n_classes)  
&ensp;&ensp;  loss = tf.nn.sigmoid_cross_entropy_with_logits(  
&ensp;&ensp;&ensp;&ensp;      labels=labels_one_hot,  
&ensp;&ensp;&ensp;&ensp;      logits=logits)  
&ensp;&ensp;  loss = tf.reduce_sum(loss, axis=1)  

__Note:__ 默认情况，会使用一个log-uniform(zipfian) distribution来采样，所以你的labels必须要按照频率的降序来排列来获得一个比较好的结果。
* zipf'law zipfian distribution:
 > 在自然语言的语料库里，一个单词出现的频率与它在频率表里的排名成反比。所以，频率最高的单词出现的频率大约是出现频率第二位的单词的2倍，而出现频率第二位的单词则是出现频率第四位的单词的2倍。这个定律被作为任何与幂定律概率分布有关的事物的参考。[link](http://www.cnblogs.com/sddai/p/6081447.html)
 
> tf.nn.log_uniform_candicate_sampler(
&ensp;&ensp;    true_classes,
&ensp;&ensp;    num_true,
&ensp;&ensp;    num_sampled,
&ensp;&ensp;    unique,
&ensp;&ensp;    range_max,
&ensp;&ensp;    seed=None,
&ensp;&ensp;    name=None
)
 >>  这个函数是从log-uniform base distribution采样一组样本。这个操作从整数范围\[0,range_max)中随机取样(sampled_candicates)样本数目的一个tensor。  
    若unique=True,则无放回采样，unique=False则又放回采样。
    这个operation的base distribution是近似log-uniform或者zipfan distribution的分布：
    $P(class) = (log(class + 2) - log(class + 1)) / log(range_max + 1)$  
    当目标类的分布例如再字典(lexicon)中表达词汇的类别式按照频率的降序来排列的，那么用这种采样方式是有用的，如果不是，那么不建议采用这种sample方式。
    另外，这个operation返回tensors __true_expected_count and sampled_expected_count__ 代表每个target classes(true_classes)和sampled classes(sampled_candidates)在sampled classes的平均张量中出现的次数。这些值对应到[candidate sampling](https://www.tensorflow.org/extras/candidate_sampling.pdf?hl=zh-cn)的$Q(y|x)$中。如果unique=True，那么这些就是post-rejection的概率，我们是近似计算他们。  
   >> 
__Args:__ 
* true_classes: A Tensor of type int64 and shape [batch_size, num_true]. The target classes.
* num_true: An int. The number of target classes per training example.
* num_sampled: An int. The number of classes to randomly sample.
* unique: A bool. Determines whether all sampled classes in a batch are unique.
* range_max: An int. The number of possible classes.
* seed: An int. An operation-specific seed. Default is 0.
* name: A name for the operation (optional).  

>>__Returns:__ 
* sampled_candidates: A tensor of type int64 and shape [num_sampled]. The sampled classes.
* true_expected_count: A tensor of type float. Same shape as true_classes. The expected counts under the sampling distribution of each of true_classes.
* sampled_expected_count: A tensor of type float. Same shape as sampled_candidates. The expected counts under the sampling distribution of each of sampled_candidates.

[log-uniformed distribution](https://stats.stackexchange.com/questions/155552/what-does-log-uniformly-distribution-mean) 是指采样的词出现的频率取log是均匀分布的：
    ![Title](../../image/log-uniform.png)

__Note:__ 当num_true>1，那么我们将每个target class的target 概率置为 1/num_true，这样target概率在每个例子中相加为1.

__Note：__ 现在目标类的数目是一定的。

In [ ]:
# Compute the NCE loss, using a sample of the negative labels each time.
loss = tf.reduce_mean(
  tf.nn.nce_loss(weights=nce_weights,
                 biases=nce_biases,
                 labels=train_labels,
                 inputs=embed,
                 num_sampled=num_sampled,
                 num_classes=vocabulary_size))

这样，我们得到了loss节点，将这个节点添加到计算gradient和更新参数上。在这里我们采用SGD。

In [ ]:
# We use the SGD optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

## Training the Model
训练模型就是用一个feed_dict将数据传递到placeholders并且调用__tf.Session.run__ 在循环中使用新的数据，简单的代码如下：

In [ ]:
for inputs, labels in generate_batch(...):
    feed_dict = {train_inputs:inputs, train_labels:labels}
    _,cur_loss = session.run([optimizer,loss],feed_dict=feed_dict)

## Visualizing the Learned Embeddings
在训练完成之后，通过t-SNE实现可视化
![Title](../../image/tsne.png)
更加复杂的word2Vec版本实现见[link](https://github.com/tensorflow/models/blob/master/tutorials/embedding/word2vec.py)

## Evaluating Embeddings: Analogical Reasoning
在NLP的大部分预测任务中embedding是比较有用的。 缺乏(short of) 训练一个全面爆发的(ful-blown) part-of-speech model 或者named-entity model，一个简单的方法来评估embedding是直接用他们来预测语法和语义的关系例如:king is to queen as father is to ?.这叫做类比推理(analogical reasoning)，这个任务在[ Mikolov and colleagues](https://www.aclweb.org/anthology/N13-1090)中又介绍，dataset下载于[download.tensorflow.org.](http://download.tensorflow.org/data/questions-words.txt?hl=zh-cn)

怎么样完成这个evaluation的详见[word2vec](https://github.com/tensorflow/models/blob/master/tutorials/embedding/word2vec.py)中的build_eval_graph()和evaluate()函数。

hyperparameters的选择队这个任务的影响很大。来达到这个任务现在所有的技术的运行结果要求这个在很大的dataset上面训练，并且仔细的调节这些参数和使用一些小技巧例如减采样(subsampling the data)数据。

## Optimizing the Implementation
在vanilla implementation中已经展示了TensorFlow的灵活性。例如，改变training 对象就如同换掉 tf.nn.nce_loss()而采用另外一种线程的tf.nn.sampled_softmax_loss()一样简单。如果你又一个loss function的新的想法，你可以自己手动在tensorflow上位这个新的目标写一个表达式，并且让optimizer来计算它的微分。这种灵活性在机器学习模型开发的探索阶段是非常有价值的，在这个阶段我们正在尝试几个不同的想法并且快速迭代。

一旦你有一个满足的模型结构，那么让你的实现更加的有效是非常值得的。例如，在tutorial原始的代码当中由于使用Python来读数据，这样让速度有所下降，因为每一个在tensorFLow的后台需要有一些小小的操作。如果说你发现你的模型在输入数据上有很大的瓶颈，你可以实现一个自定义的数据读取，详见[Custom Data Readers](https://www.tensorflow.org/extend/new_data_formats?hl=zh-cn),参见例子[tutorials/embedding/word2vec.py](https://github.com/tensorflow/models/blob/master/tutorials/embedding/word2vec.py)。
还有[adding a new op](https://www.tensorflow.org/extend/adding_an_op?hl=zh-cn),参见例子[word2vec_optimized.py](https://github.com/tensorflow/models/blob/master/tutorials/embedding/word2vec_optimized.py)。

In [1]:
-1%5

4